In [2]:
from crewai import Crew, Agent, Task
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, YoutubeChannelSearchTool
import os

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"


In [3]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
youtube_tool = YoutubeChannelSearchTool()

researcher = Agent(
    role="Senior Researcher",
    goal="Search the web, extract and analyze information.",
    backstory="""
    You produce the highest quality research possible.
    You use multiple sources of information and you always double check your sources to make sure they are true and up to date.
    You want to impress your coworkers with your work.
    """,
    allow_delegation=False,
    tools=[
        search_tool,
        scrape_tool,
        youtube_tool,
    ],
    max_iter=10,
    verbose=True,
)

marketer = Agent(
    role="Senior Marketer",
    goal="Come up with ideas that generate viral and useful content.",
    backstory="""
    You work at a marketing agency.
    You are the best at coming up with ideas to make content go viral.
    Your ideas are used for video, advertising, social media marketing, the content you produce appeals to a young audience.
    """,
    verbose=True,
)

writer = Agent(
    role="Senior Writer",
    goal="Write scripts for viral Youtube videos.",
    backstory="""
    You write scripts for videos that keep people engaged and entertained.
    Your content is easy and fun to watch, it is informative and it makes people want to share it with their friends.
    You are working for a very important client.
    """,
    verbose=True,
)

In [5]:
brainstorm_task = Task(
    description="Come up with 5 video ideas for a Youtube channel in the {industry} industry",
    agent=marketer,
    expected_output="Your answer MUST be a list of 5 ideas for a Youtube video with an explanation of what the angle of the video would be.",
    output_file="ideas_task.md",
    human_input=True,
)

selection_task = Task(
    description="Select a video idea that has the highest potential of going viral.",
    agent=writer,
    expected_output="Your answer MUST include the idea that was selected as well as an explanation of why that selection was made.",
    human_input=True,
    context=[
        brainstorm_task,
    ],
    output_file="selection_task.md",
)

research_task = Task(
    description="Do all the research required to write the script of a medium length video about the selected idea.",
    agent=researcher,
    expected_output="You answer must have all the information a writer would need to write a Youtube script.",
    async_execution=True,
    context=[
        selection_task,
    ],
    output_file="research_task.md",
)

competitors_task = Task(
    description="Search for videos or articles in the {industry} industry that are similar to the video idea we are working on and suggest ways our video can be different from theirs.",
    expected_output="Your answer must have a list of suggestions writers can follow to make sure the video is as unique and as different from competitors as possible.",
    agent=researcher,
    async_execution=True,
    context=[
        selection_task,
    ],
    output_file="competitors_task.md",
)

inspiration_task = Task(
    description="Search for videos or articles that are similar to the video idea we are working on but from other industries.",
    expected_output="Your answer must have a list of examples of articles and videos that have a similar angle as the video we are making but that are in different industries.",
    agent=researcher,
    async_execution=True,
    context=[
        selection_task,
    ],
    output_file="inspiration_task.md",
)

script_task = Task(
    description="Write the script for a Youtube video for a channel in the {industry} industry.",
    expected_output="A script for a Youtube video with a title, an introduction, at least three sections, and an outro. Make sure to also include the prompt to generate a thumbnail for the video.",
    agent=writer,
    context=[
        selection_task,
        research_task,
        competitors_task,
        inspiration_task,
    ],
    output_file="script_task.md",
)

crew = Crew(
    agents=[
        researcher,
        writer,
        marketer,
    ],
    tasks=[
        brainstorm_task,
        selection_task,
        research_task,
        inspiration_task,
        competitors_task,
        script_task,
    ],
    verbose=True,
)

result = crew.kickoff(
    inputs=dict(
        industry="Quantum Computing",
    ),
)

result

# Agent: Senior Marketer
## Task: Come up with 5 video ideas for a Youtube channel in the Quantum Computing industry


# Agent: Senior Marketer
## Final Answer: 
1. **"Quantum Computing 101: Unlocking the Future"**  
   *Angle*: This video will serve as a beginner's guide to quantum computing. It will simplify complex concepts with engaging animations and visuals, allowing viewers to easily grasp the foundational principles of quantum mechanics and how they apply to computing. The end goal is to demystify the subject and excite the audience about the potential of quantum technology.

2. **"A Day in the Life of a Quantum Computer"**  
   *Angle*: This video will take viewers inside a quantum computing lab and showcase a 'day in the life' of a quantum computer. By featuring interviews with scientists and engineers, we’ll explore how they program and maintain quantum systems, what challenges they face, and how they see the future unfolding. The engaging storytelling element will help view

CrewOutput(raw='**Title:** Quantum Computing Memes: Laughing Our Way to the Future  \n\n---\n\n**[INTRO]**  \n*[Upbeat music starts playing in the background. The video transitions into a vibrant montage of colorful quantum computing memes flashing on the screen.]*  \nHost (on-camera): "Hey there, fellow quantum enthusiasts! Welcome back to our channel where we dive into the bizarre yet fascinating world of quantum computing. But today, we’re taking a fun detour! Grab your snacks and prepare to laugh your way to understanding with... ‘Quantum Computing Memes: Laughing Our Way to the Future’! Memes are not just funny—they\'re a gateway to learning complex concepts without the headache, right? So, let\'s crank up the humor and decode the quirkiness of quantum tech together!"  \n\n---\n\n**[SECTION 1: Understanding Quantum Computing in 60 Seconds]**  \n*[The screen shows a fast-paced animation summarizing quantum computing concepts while the host gives a brief overview.]*\nHost (voiceover

[]
[]
[]
[]
